In [64]:
import numpy as np
from tqdm import tqdm

In [121]:
# generate mask from image
from PIL import Image

def generate_mask(img, nz, nx):
  im = Image.open(img).convert("L")
  if im.size != (nx-2, nz-2):
    print("image size different. Resized")
    im = im.resize((nx-2, nz-2))

  arr = np.array(im)

  mask = np.where(arr < 128, 0, 1)
  return mask

In [145]:
# 3d explicit method
nx = 32
ny = 12
nz = 12
nt = 1000

lx = 3
ly = 1
lz = 1
t = 1

dx = lx/(nx-1)
dy = ly/(ny-1)
dz = lz/(nz-1)
dt = t/nt
D = 0.01
mask = generate_mask("random_dots.png", nz, nx)

x = np.linspace(0.0, lx, nx)
y = np.linspace(0.0, ly, ny)
z = np.linspace(0.0, lz, nz)
c0 = 1

# Initial condition
C = np.zeros((nz, ny, nx))
for z in range(1, nz-1):
  for x in range(1, nx-1):
    C[z, -1, x] = c0 * (1-mask[z-1, x-1])

# Calculate
for i in tqdm(range(nt)):
  for j in range(1, nz-1):
    for k in range(1, ny-1):
      for l in range(1, nx-1):
        # Calculate at C(j, k, l, t+1)
        C[j, k, l] = C[j, k, l] + D*(
            (dt/dx**2)*(C[j, k, l+1] - 2*C[j, k, l] + C[j, k, l-1]) +
            (dt/dy**2)*(C[j, k+1, l] - 2*C[j, k, l] + C[j, k-1, l]) +
            (dt/dz**2)*(C[j+1, k, l] - 2*C[j, k, l] + C[j-1, k, l])
            )
  # Set boundary condition
  C[0, :, :] = C[1, :, :]
  C[-1, :, :] = C[-2, :, :]
  C[:, 0, :] = C[:, 1, :]
  C[:, :, 0] = C[:, :, 1]
  C[:, :, -1] = C[:, :, -2]

  for z in range(1, nz-1):
    for x in range(1, nx-1):
      if (mask[z-1, x-1] == 0):
        C[z, -1, x] = c0
      else:
        C[z, -1, x] = C[z, -2, x]

100%|██████████| 1000/1000 [00:15<00:00, 66.45it/s]


In [133]:
# Load CSV
C_load = np.loadtxt("16x16x16.csv", delimiter=",")
C_Params = C_load[-10:]
C_Values = C_load[:-10].reshape(int(C_Params[0]), int(C_Params[1]), int(C_Params[2]))

In [137]:
mse_value = np.mean(np.square(C_Values - C))
print(f"Mean Squared Error (MSE) between C_ADI and C: {mse_value}")

Mean Squared Error (MSE) between C_ADI and C: 1.378312278714684e-07


In [146]:
import csv
flattened = np.append(C.flatten(), [nx, ny, nz, D, t, dt, lx, ly, lz, c0])

# Save as a single comma-separated row in a CSV
with open('flattened_3d_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(flattened)

print("3D data flattened and saved to 'flattened_3d_data.csv'")

3D data flattened and saved to 'flattened_3d_data.csv'
